# Overall performance of module detection methods

In [1]:
import sys
import os
# sys.path.insert(0,os.path.abspath("../lib/"))

import json

from util import JSONExtendedEncoder

%load_ext autoreload
%autoreload 2

%matplotlib inline
from matplotlib.pyplot import *

import pandas as pd
import numpy as np

import multiprocessing as mp

from itertools import product

import itertools
import shutil

import os

conf_folder = "conf/"

# Settings

In [2]:
n_jobs = int(mp.cpu_count() - 1)
method_name = None

In [3]:
# Parameters
method_name = "agglom_pearson_abs"
n_jobs = 1


In [4]:
assert method_name is not None, "You have to specify a method_name"

In [5]:
print(f"Using {n_jobs} cores")

Using 1 cores


# Running a method on different parameter settings and datasets

Note: If you downloaded the results from zenodo, you don't need to rerun this for "dummy", "agglom", "ica_zscore", "spectral_biclust" and "meanshift"

The following code will explore the parameters of a module detection method on every dataset using a grid-search approach.

If you want to run your own method, you should wrap it into a python function and add its parameters to `conf/paramexplo_blueprints.py`. We will show the whole workflow here for a "dummy"  (but fast) clustering method, which will simply group genes randomly.

Every module detection method is wrapped in a python function (see `scripts/moduledetection.py`)

Because module detection methods usually take a while to run, we generate the files necessary to run a method on the several parameter settings and datasets here. These can then be easily called from the commandline, for example on a computer cluster or locally using GNU `parallel`.

This function will be called by scripts/moduledetection.py , which will save the modules in the correct format along with additional run information (such as running times).

In [6]:
# datasets to run
datasetnames = [
    "ecoli_colombos",
    "ecoli_dream5",
    "yeast_gpl2529",
    "yeast_dream5",
    "synth_ecoli_regulondb",
    "synth_yeast_macisaac",
    "human_tcga",
    "human_gtex",
    "human_seek_gpl5175",
    "ecoli_precise2"
]

# choose the method to evaluate
# method_name = "agglom_pearson_abs" # use the dummy method to check if everything works correctly
# method_name = "agglom" # this method runs very fast, and has the best performance among clustering methods
# method_name = "ica_zscore" # this method runs very slow, but has approx. the highest performance in the benchmark
# method_name = "spectral_biclust" # top biclustering method
# method_name = "meanshift"

To add your own method, create a function with "your_method_name" in the `lib/clustering.py` file (or any other file as long as it's imported in `scripts/moduledetection.py`.
This function should accept an `E` object (which is a dataframe with genes in columns) and any additional parameters
Then add reasonable parameter setting of your method to `conf/paramexplo_blueprints.py`.

method_name = "your_method_name"

In [7]:
# paramexplo_blueprints.py stores for every method the parameters which will be varied using a grid-search approach.
%run ../conf/paramexplo_blueprints.py
methodblueprint = blueprints[method_name]

In [8]:
methodblueprint

{'staticparams': {'method': 'agglom_pearson_abs',
  'simdist_function': 'pearson_correlation_absolute'},
 'dynparams': {'linkage': ['complete', 'average'],
  'k': array([ 25.,  50.,  75., 100., 125., 150., 175., 200., 225., 250., 275.,
         300.])},
 'type': 'moduledetection'}

Generate different parameter settings using a grid-search.

In [9]:
params_folder = "conf/paramexplo/" + method_name + "/"
if os.path.exists("../" + params_folder):
    shutil.rmtree("../" + params_folder)
os.makedirs("../" + params_folder)

methodsettings = []
method_locations = []
i = 0
for dynparam_combination in list(itertools.product(*[methodblueprint["dynparams"][param] for param in sorted(methodblueprint["dynparams"].keys())])):
    method = {"params":{}}
    method["params"] = methodblueprint["staticparams"].copy()
    method["params"].update(dict(zip(sorted(methodblueprint["dynparams"].keys()), dynparam_combination)))
    method["location"] = params_folder + str(i) + ".json"
    method["seed"] = 0

    methodsettings.append(method)

    json.dump(method, open("../" + method["location"], "w"), cls=JSONExtendedEncoder)

    method_locations.append(method["location"])

    i+=1

Now combine the different parameter settings and datasets. Then generate the different python commands to run every parameter setting and dataset in parallel.

In [10]:
settings_name = "paramexplo/{method_name}".format(method_name = method_name)
settings = []
for datasetname in datasetnames:
    for setting_ix, methodsetting in enumerate(methodsettings):
        settingid = datasetname + "_" + str(setting_ix)
        settings.append({
            "dataset_location":"conf/datasets/" + datasetname + ".json",
            "dataset_name":datasetname,
            "method_location":methodsetting["location"],
            "output_folder":"results/" + methodblueprint["type"] + "/{settings_name}/{settingid}/".format(settings_name=settings_name, settingid=settingid),
            "settingid":settingid
        })
json.dump(settings, open("../conf/settings/{settings_name}.json".format(settings_name=settings_name), "w"))

In [11]:
settings_dataset = pd.DataFrame([dict(settingid=setting["settingid"], **json.load(open("../" + setting["dataset_location"]))["params"]) for setting in settings])
settings_method = pd.DataFrame([dict(settingid=setting["settingid"], **json.load(open("../" + setting["method_location"]))["params"]) for setting in settings])

In [12]:
# commands = ""
# for i, setting in enumerate(settings):
#     #commands += "python scripts/moduledetection.py {method_location} {dataset_location} {output_folder} 0 test\n".format(**setting)
#     commands += "python3 scripts/" + methodblueprint["type"] + ".py {method_location} {dataset_location} {output_folder}\n".format(**setting)

# commands_location = "tmp/{settings_name}.txt".format(**locals())
# os.makedirs("../" + os.path.dirname(commands_location), exist_ok=True)
# with open("../" + commands_location, "w") as outfile:
#     outfile.write(commands)
# commands_location = "tmp/{settings_name}.txt".format(**locals())
# os.makedirs(os.path.dirname("../tmp/" + commands_location), exist_ok=True)
# with open("../tmp/" + commands_location, "w") as outfile:
#     outfile.write(commands)
    
# #script_location = generate_batchcode(commands_location, settings_name, len(settings), {"memory":"10G", "numcores":1}, "biclust_comp2")

# # this command can be used on most linux computers to run the different parameter settings in parallel
# print("parallel -j 4 -a " + commands_location)

# Evaluating the method

In [13]:
from modulescomparison import ModevalKnownmodules, ModevalCoverage

Note: If you downloaded the results from zenodo, you don't need to rerun this for "dummy", "agglom", "ica_zscore", "spectral_biclust" and "meanshift"

## By comparing with known modules

Evaluate by comparing with known modules

In [14]:
# create pool of processors
if "pool" in locals().keys():
    pool.close()
pool = mp.Pool(n_jobs)

In [15]:
settings_filtered = [setting for setting in settings if not setting["dataset_name"].startswith("human")] # only evaluate non-human datasets
modeval = ModevalKnownmodules(settings_filtered, baseline = True)

In [16]:
modeval.run(pool)
modeval.save(settings_name)

In [17]:
modeval.load(settings_name)

In [18]:
modeval.scores

,recovery,relevance,F1rr,recall,precision,F1rp,F1rprr,F1rr_permuted,F1rp_permuted,F1rprr_permuted,settingid,knownmodules_name,regnet_name,goldstandard,runningtime
0,0.226586,0.177894,0.199309,0.051668,0.066232,0.058051,0.089913,2.700201,8.357507,4.081668,ecoli_colombos_0,mcl2,ecoli_regulondb,ecoli_regulondb#mcl2,0.838243
1,0.203340,0.143121,0.167998,0.052379,0.038628,0.044465,0.070318,2.284497,5.274020,3.188055,ecoli_colombos_0,minimal,ecoli_regulondb,ecoli_regulondb#minimal,0.838243
2,0.162774,0.114664,0.134547,0.021337,0.039583,0.027727,0.045979,1.790495,3.954100,2.464855,ecoli_colombos_0,ap3,ecoli_regulondb,ecoli_regulondb#ap3,0.838243
3,0.200813,0.163221,0.180076,0.041041,0.048309,0.044379,0.071209,2.695969,6.569270,3.823009,ecoli_colombos_0,tc1,ecoli_regulondb,ecoli_regulondb#tc1,0.838243
4,0.204883,0.163406,0.181809,0.038667,0.051042,0.044001,0.070854,2.855823,8.207359,4.237256,ecoli_colombos_0,mcl3,ecoli_regulondb,ecoli_regulondb#mcl3,0.838243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2131,0.283008,0.187216,0.225355,0.137420,0.031092,0.050710,0.082791,3.063771,6.020099,4.060869,ecoli_precise2_19,minimal,ecoli_regulondb,ecoli_regulondb#minimal,1.338355
2132,0.257532,0.266224,0.261806,0.153174,0.023756,0.041133,0.071096,3.565290,4.884048,4.121754,ecoli_precise2_20,minimal,ecoli_regulondb,ecoli_regulondb#minimal,1.232296
2133,0.276771,0.194246,0.228279,0.138249,0.029967,0.049256,0.081029,3.104193,5.847624,4.055524,ecoli_precise2_21,minimal,ecoli_regulondb,ecoli_regulondb#minimal,1.468318
2134,0.267568,0.272469,0.269997,0.155421,0.022837,0.039823,0.069408,3.676621,4.728512,4.136745,ecoli_precise2_22,minimal,ecoli_regulondb,ecoli_regulondb#minimal,1.426097


## Using the coverage of regulators

In [19]:
# create pool of processors
if "pool" in locals().keys():
    pool.close()
pool = mp.Pool(n_jobs)

In [20]:
settings_filtered = [setting for setting in settings if setting["dataset_name"].startswith("human")] # only evaluate human datasets
modeval = ModevalCoverage(settings_filtered, baseline = True)

In [21]:
modeval.run(pool)
modeval.save(settings_name)

Evaluating a total of 72 settings.


In [22]:
modeval.load(settings_name)

In [23]:
modeval.scores

,aucodds,aucodds_permuted,settingid,goldstandard,runningtime
0,0.095364,3.598500,human_tcga_0,regcircuit,5.915454
1,0.025903,0.977450,human_tcga_1,regcircuit,5.706284
2,0.107492,4.056172,human_tcga_2,regcircuit,4.702357
3,0.042776,1.614130,human_tcga_3,regcircuit,5.028560
4,0.110955,4.186826,human_tcga_4,regcircuit,4.825297
...,...,...,...,...,...
67,0.166105,6.267905,human_seek_gpl5175_19,regcircuit,2.194216
68,0.228174,8.610032,human_seek_gpl5175_20,regcircuit,1.838537
69,0.166468,6.281577,human_seek_gpl5175_21,regcircuit,2.168797
70,0.211450,7.978940,human_seek_gpl5175_22,regcircuit,2.256483


**TODO:** I understand that the previous code generates some files that will be used later to create the final dataframe with scores and the plots.